In [1]:
import json
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Bidirectional, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

file = open('intents.json', 'r')
data = json.load(file)
file.close()

patterns = []
responses = []
labels = []

for intent in data["intents"]:
    if "patterns" in intent and "responses" in intent:
        for pattern in intent["patterns"]:
            patterns.append(pattern)
            responses.append(np.random.choice(intent["responses"]))
        labels.extend([intent["tag"]] * len(intent["patterns"]))

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 30
input_ids = []
attention_masks = []

for pattern in patterns:
    encoded = tokenizer.encode_plus(pattern, add_special_tokens=True, max_length=max_length, padding='max_length', truncation=True, return_attention_mask=True)
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])

input_ids = np.array(input_ids)
attention_masks = np.array(attention_masks)
labels_encoded = np.array(labels_encoded)

X_train, X_test, y_train, y_test = train_test_split(input_ids, labels_encoded, test_size=0.2, random_state=42)

model = Sequential()
model.add(Input(shape=(max_length,)))
model.add(Embedding(input_dim=tokenizer.vocab_size, output_dim=256))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(128, dropout=0.3)))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, mode='min')
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping, reduce_lr, model_checkpoint])

user_input = input("You: ")
encoded_input = tokenizer.encode_plus(user_input, add_special_tokens=True, max_length=max_length, padding='max_length', truncation=True, return_attention_mask=True)
input_ids = np.array(encoded_input['input_ids']).reshape(1, -1)
attention_mask = np.array(encoded_input['attention_mask']).reshape(1, -1)
prediction = model.predict(input_ids)
intent_index = np.argmax(prediction)
predicted_intent = label_encoder.inverse_transform([intent_index])[0]
response = np.random.choice([resp for resp, lbl in zip(responses, labels) if lbl == predicted_intent])
print("Bot:", response)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Epoch 1/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 89s 578ms/step - accuracy: 0.5404 - loss: 2.0822 - val_accuracy: 0.4652 - val_loss: 2.5838 - learning_rate: 0.0010
Epoch 2/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 79s 561ms/step - accuracy: 0.8986 - loss: 0.4494 - val_accuracy: 0.7861 - val_loss: 0.9930 - learning_rate: 0.0010
Epoch 3/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 79s 559ms/step - accuracy: 0.9457 - loss: 0.2245 - val_accuracy: 0.9046 - val_loss: 0.4040 - learning_rate: 0.0010
Epoch 4/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 82s 563ms/step - accuracy: 0.9607 - loss: 0.1529 - val_accuracy: 0.9091 - val_loss: 0.3688 - learning_rate: 0.0010
Epoch 5/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 80s 551ms/step - accuracy: 0.9802 - loss: 0.0878 - val_accuracy: 0.9207 - val_loss: 0.3463 - learning_rate: 0.0010
Epoch 6/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 90s 604ms/step - accuracy: 0.9896 - loss: 0.0519 - val_accuracy: 0.9207 - val_loss: 0.3338 - learning_rate: 0.0010
Epoch 7/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 147s 640ms/step - accur